In [0]:
# Make sure upload below before run
# upload models.py torch_utils.py helper.py preprocess.pkl


In [0]:
import helper
import numpy as np
import torch
import torch.nn as nn
from torch_utils import batch_data, train_rnn
from models import RNN
import time
import os
# hyperparameters
sequence_length = 6
batch_size = 128
num_epochs = 10
learning_rate = 0.002
embedding_dim = 256
hidden_dim = 256
n_layers = 2
show_every_n_batches = 500

In [2]:
# def load_preprocess():
#     """
#     Load the Preprocessed Training data and return them in batches of <batch_size> or less
#     """
#     return pickle.load(open('preprocess.pkl', mode='rb'))
import os
import importlib
importlib.reload(helper)

<module 'helper' from '/content/helper.py'>

In [0]:

# load data
int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
train_loader = batch_data(int_text, sequence_length, batch_size)
vocab_size = len(vocab_to_int)
output_size = len(vocab_to_int)



In [0]:
# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')



In [0]:
# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.25)
if train_on_gpu:
    rnn.cuda()

In [6]:
# set decay, optimizer, and loss
decay_rate = learning_rate / num_epochs
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
# optimizer = torch.optim.SGD(rnn.parameters(), lr=learning_rate, momentum=0.9, weight_decay=decay_rate)
criterion = nn.CrossEntropyLoss()

# train the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, train_loader, show_every_n_batches)


Training for 10 epoch(s)...
epoch  1
Epoch:    1/10    Loss: 5.907130914688111 Decrease Rate: inf 

Model Trained and Saved
Epoch:    1/10    Loss: 5.1510430479049685 Decrease Rate: 0.7560878667831421 

Model Trained and Saved
Epoch:    1/10    Loss: 4.940020363807678 Decrease Rate: 0.21102268409729064 

Model Trained and Saved
Epoch:    1/10    Loss: 4.830107000350952 Decrease Rate: 0.10991336345672575 

Model Trained and Saved
Epoch:    1/10    Loss: 4.775876293182373 Decrease Rate: 0.05423070716857925 

Model Trained and Saved
Epoch:    1/10    Loss: 4.714502788066864 Decrease Rate: 0.0613735051155091 

Model Trained and Saved
Epoch:    1/10    Loss: 4.65915013885498 Decrease Rate: 0.05535264921188343 

Model Trained and Saved
Epoch:    1/10    Loss: 4.620100061416626 Decrease Rate: 0.03905007743835398 

Model Trained and Saved
Epoch:    1/10    Loss: 4.581644585609436 Decrease Rate: 0.03845547580719 

Model Trained and Saved
Epoch:    1/10    Loss: 4.535595349311828 Decrease Rate: 

In [0]:
# If you have a pretrained model, you can start from here,
# make sure load helper.py trained_rnn_new.pt
import torch
import helper
import torch.nn.functional as F
import time

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    train_on_gpu = torch.cuda.is_available()

    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    print("good 1")

    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)

        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))

        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)

        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu

        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()

        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())

        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)

        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i

    gen_sentences = ' '.join(predicted)

    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')

    # return all the sentences
    return gen_sentences

In [0]:
_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
# if you are running in uploaded model
trained_rnn = helper.load_model('./trained_rnn_new')

In [10]:
# import pickle
# with open('preprocess.pkl', 'rb') as f:
#     data = pickle.load(f)
# print(data[2])

{0: 'cornucopia', 1: 'form', 2: 'passion', 3: 'referring', 4: 'pajamas', 5: 'weddings', 6: 'deterministic', 7: 'assumed', 8: 'quieter', 9: 'auditioning', 10: 'able', 11: 'depressing', 12: 'las', 13: 'sunscreen', 14: 'sate', 15: 'intelligent', 16: 'helmets', 17: 'puzzling', 18: 'reductio', 19: 'highlights', 20: 'mentally', 21: 'youngsters', 22: 'ager', 23: 'motorized', 24: 'microsoft', 25: 'flirtations', 26: 'spirited', 27: 'grapes', 28: 'uninterruptible', 29: 'created', 30: 'fan', 31: 'irritate', 32: 'varicose', 33: 'americans', 34: 'lifted', 35: 'moth', 36: 'tragedy', 37: 'absurdum', 38: 'spreads', 39: 'when', 40: 'slave', 41: 'deserved', 42: 'punch', 43: 'pieces', 44: 'arbitrary', 45: 'distractions', 46: 'kenneth', 47: 'favorite', 48: 'mall’s', 49: 'plummeting', 50: 'tortured', 51: 'inherently', 52: 'samantha', 53: 'an', 54: 'michelle', 55: 'panelists', 56: 'what', 57: 'lookin’', 58: 'comparable', 59: 'crime', 60: 'sniffed', 61: 'tools', 62: 'provides', 63: 'curtis', 64: 'broaden', 6

In [19]:
# run the cell multiple times to get different results!\
import datetime
gen_length = 400
prime_word = 'magically' # name for starting the script
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ''], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
# generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100)
print(generated_script)


f =  open("generated_script" + str(datetime.datetime.now()) + ".txt","w")
f.write(generated_script)
f.close()

good -1
good 0
good 1
magically posting an hour ago ||PERIOD|| i was going out there ||PERIOD|| penny ||COLON|| oh ||COMMA|| no ||PERIOD|| howard ||COLON|| oh ||PERIOD|| well ||COMMA|| i don’t think you could do that ||COMMA|| but i was thinking of the math ||COMMA|| so let’s see ||COMMA|| i’m not gonna go to work ||PERIOD|| sheldon ||COLON|| oh ||COMMA|| no ||COMMA|| i’m not gonna get to the party ||PERIOD|| leonard ||COLON|| oh ||COMMA|| i guess ||PERIOD|| howard ||COLON|| i don’t know ||PERIOD|| i was hoping i could do it ||PERIOD|| howard ||COLON|| you can’t have to do it again ||PERIOD|| penny ||COLON|| yeah ||COMMA|| well ||COMMA|| i know ||PERIOD|| leonard ||COLON|| you know what ||QUESTION_MARK|| i think it’s like a bobcat ||COMMA|| and i was wondering if i could just get it ||PERIOD|| sheldon ||COLON|| well ||COMMA|| i don’t want to see you ||PERIOD|| howard ||QUESTION_MARK|| sheldon ||COLON|| oh ||COMMA|| i know ||PERIOD|| sheldon ||COLON|| no ||PERIOD|| i mean ||COMMA|| i do